In [1]:
import requests
import json
import pandas as pd
from pandas import json_normalize
import sqlite3
import numpy as np

cmoneyWEB = 'https://m.moneydj.com/jsondata/djjson/FundJsonData.xdjjson?x=XDT003F'

def 每日新聞():
    global df2
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36 Edg/105.0.1343.33'}
    res = requests.get(cmoneyWEB, headers = headers)
    res.encoding = 'utf-8' # requests.models.Response
    # json相關(https://medium.com/datainpoint/scraping-json-with-python-requests-95a56e61096a)
    res_text = res.text # str
    today_json = json.loads(res_text) # dict
    
    # json_normalize()函式被非常廣泛地用於讀取巢狀的json字串並返回一個DataFrame
    df = json_normalize(today_json['ResultSet']['Result'])
    df2 = df.copy()
    df2 = df2.replace('\.[A-Z]*','',regex=True) # 用正則表達式刪除.TW
    df2 = df2.replace('\([^\x00-\xff]*\)','',regex=True) # 正則表達式刪除來源
    df2 = df2.loc[:,('XQID','V3','V4','V1')]
    df2.columns = ['代碼','中文簡稱','新聞內容','日期']
    print(df2)



if __name__ == '__main__':
    每日新聞()

      代碼    中文簡稱                新聞內容          日期
0   1590  亞德客-KY      亞德客-KY擴產能 營運看升  2023/06/21
1   2303      聯電  台積/聯電等  入列麥格理十大首選股  2023/06/21
2   2317      鴻海     鴻海旗下鴻佰 將建太空運算產線  2023/06/21
3   2330     台積電  台積/聯電等  入列麥格理十大首選股  2023/06/21
4   2340      台亞   台亞布局寬能隙半導體 擴建銅鑼新廠  2023/06/21
5   2360      致茂  台積/聯電等  入列麥格理十大首選股  2023/06/21
6   2449    京元電子  台積/聯電等  入列麥格理十大首選股  2023/06/21
7   2454     聯發科  台積/聯電等  入列麥格理十大首選股  2023/06/21
8   2634      漢翔       漢翔傳捷報 奪逾百億元大單  2023/06/21
9   3044      健鼎    健鼎三應用需求強勁 營運可望升溫  2023/06/21
10  3711   日月光投控  台積/聯電等  入列麥格理十大首選股  2023/06/21
11  5203      訊連        訊連衝刺業績 三引擎熱轉  2023/06/21
12  5258      虹堡           虹堡：拉貨力道浮現  2023/06/21
13  5274      信驊  台積/聯電等  入列麥格理十大首選股  2023/06/21
14  5284  jpp-KY    jpp-KY拚擴產 業績倒吃甘蔗  2023/06/21
15  6230   尼得科超眾          尼得科超眾 營運看旺  2023/06/21
16  6290      良維          良維 訂單將逐季向上  2023/06/21
17  6414      樺漢  台積/聯電等  入列麥格理十大首選股  2023/06/21
18  6669      緯穎  台積/聯電等  入列麥格理十大首選股  2023/06/21
19  6689     伊雲谷  台積

In [2]:
# 下載財報
import requests

res = requests.get('https://m.moneydj.com/jsondata/djjson/FundJsonData.xdjjson?x=XDT003F')
res.encoding = 'big-5'


lines = res.text.split("XQID")

import pandas as pd
from io import StringIO


dfs = pd.read_csv(StringIO("\n".join(lines).replace(':',''))) # \n為換行的意思,把list轉換為DataFrame，另外把“=”轉換成‘’


dfs.drop(['StatusCode0','Comment"報上有利多個股','FieldLength4','Result[{"'],axis=1,inplace=True)
dfs.columns=['代碼','商品','利多理由']
dfs['代碼'] = dfs['代碼'].apply(lambda x: x.split('.TW"')[0])
dfs['商品'] = dfs['商品'].apply(lambda x: x.split('V3"')[1])
dfs['利多理由'] = dfs['利多理由'].apply(lambda x: x.split('V4"')[1])
dfs['利多理由'] = dfs['利多理由'].apply(lambda x: x.split('}')[0])
dfs = dfs.set_index('代碼')
dfs

,商品,利多理由
代碼,,
1590,亞德客-KY,亞德客-KY擴產能 營運看升(經濟日報)
2303,聯電,台積/聯電等 入列麥格理十大首選股(經濟日報)
2317,鴻海,鴻海旗下鴻佰 將建太空運算產線(經濟日報)
2330,台積電,台積/聯電等 入列麥格理十大首選股(經濟日報)
2340,台亞,台亞布局寬能隙半導體 擴建銅鑼新廠(電子時報)
2360,致茂,台積/聯電等 入列麥格理十大首選股(經濟日報)
2449,京元電子,台積/聯電等 入列麥格理十大首選股(經濟日報)
2454,聯發科,台積/聯電等 入列麥格理十大首選股(經濟日報)
2634,漢翔,漢翔傳捷報 奪逾百億元大單(經濟日報)
